In [ ]:
%pip install tk

In [40]:
from deep_translator import GoogleTranslator


In [42]:
translated = GoogleTranslator(source='english', target='hindi').translate_file('Plag.txt')

In [43]:
print(translated)

प्रदूषण के प्रभावों को समझना चुनौतीपूर्ण है।
यह प्रकृति, मनुष्य, जानवरों को प्रभावित कर सकता है और प्रतिकूल जलवायु परिवर्तन में महत्वपूर्ण भूमिका निभा सकता है।
प्राकृतिक संसाधनों का ह्रास और आवासों को नष्ट करना प्रदूषण के महत्वपूर्ण प्रभाव हैं।
धुएं में सांस लेने या दूषित भोजन का सेवन करने से प्रदूषण इंसानों को नुकसान पहुंचा सकता है।
इसके अलावा, यह अस्थमा, कैंसर और हृदय रोग जैसी स्वास्थ्य समस्याओं का कारण बन सकता है।
यह पौधों, जानवरों और प्राकृतिक आवासों को नष्ट करके प्रकृति को भी प्रभावित करता है।
एक बार जब बच्चे प्रदूषण के अर्थ और प्रभावों को समझ गए हैं, तो आप उन्हें प्रदूषण पर एक निबंध लिखने के लिए कह सकते हैं।
प्रदूषण सर्वव्यापी है और इसने पर्यावरण को हर तरह से प्रभावित किया है।
पानी हो, मिट्टी हो या हवा, पर्यावरण का हर तत्व प्रदूषित है।
औद्योगिक कचरे को नदियों और जलाशयों में छोड़े जाने से जल प्रदूषित होता है।
पानी दूषित होने पर पीने के पानी की आपूर्ति कम हो जाती है।
गैर-बायोडिग्रेडेबल तत्वों को डंप करके मिट्टी प्रदूषित होती है, जिसके परिणामस्वरूप विषाक्तता होती है।
हानिकारक गैसों 

In [67]:

A="""Do you know what the Mexicans say about the Pacific?
    They say it has no memory.
    That is the place I wanna live the rest of my life;
    a warm place with no memory."""

B="They say it"

In [114]:
import numpy as np
import re
import nltk
from numpy.linalg import norm

In [69]:
def preProcess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = re.sub(r'\[.*?\]|\{.*\}', '', text)

    return text


In [ ]:
# Downloading the vocabulary of entire English language
!python -m spacy download en_core_web_md

In [ ]:
# Downloading the vocabulary of entire English language
!python -m spacy download en_core_web_lg

In [101]:
import spacy

In [119]:
# Load the language library
nlp = spacy.load('en_core_web_lg')

In [120]:
# The vector representation of the paragraph A
vecA = nlp(A)

In [121]:
# The vector representation of the paragraph B
vecB = nlp(B)

In [122]:
# Creating 2D matrix of vectors of 3 words (Each Vector is of 300 dimensions so for 3 words 2D matrix would be 3x300)
X = [vecB[0].vector,vecB[1].vector,vecB[2].vector]
X=np.array(X)

In [123]:
# Doing the same for random test words
Y = [nlp(u'He').vector,nlp(u'says').vector,nlp(u'it').vector]
Y=np.array(Y)

In [124]:
# Computing the Cosine Similartity between the corresponding 3 words
cosSim = np.sum(X*Y, axis=1)/(norm(X, axis=1)*norm(Y, axis=1))

In [128]:
print(f'They-He={cosSim[0]}, say-says={cosSim[1]}, it-it={cosSim[2]}')

They-He=0.665925145149231, say-says=0.6594836711883545, it-it=1.0
